In [1]:
import json

with open('arraysPartidasResultado.json') as file:
    data_results = json.load(file)

with open('caracteristicas_campeones.json') as file:
    data_champions = json.load(file)



In [2]:
import pandas as pd

# Transformamos la lista de campeones en un diccionario para facilitar el acceso a sus características por clave
champions_features = {champ['Key']: champ for champ in data_champions}
# Calculamos las medias de las características
features_to_average = ['Difficulty', 'HP', 'Armor', 'HPRegen', 'AttackDamage', 'AttackSpeed']
average_features = {feature: sum(champ[feature] for champ in champions_features.values()) / len(champions_features) for feature in features_to_average}

combined_data = []

# Iteramos sobre cada partida en data_results
for match in data_results:
    # Extrayendo el resultado del partido y los IDs de los campeones
    match_result = match[0]
    champion_ids = match[1:]

    # Lista para almacenar las características combinadas de los campeones de la partida
    match_features = []

    # Sumar características de todos los campeones en la partida
    for champ_id in champion_ids:
        # Si el campeón está en nuestro diccionario de características, agregamos sus datos
        if champ_id in champions_features:
            champ_data = champions_features[champ_id]
            match_features.extend([
                champ_data['Difficulty'],
                champ_data['HP'],
                champ_data['Armor'],
                champ_data['HPRegen'],
                champ_data['AttackDamage'],
                champ_data['AttackSpeed']
            ])
        else:

            # Usamos las medias si el campeón no está en el diccionario
            match_features.extend([average_features[feat] for feat in features_to_average])

    # Agregamos el resultado del partido y las características acumuladas a los datos combinados
    combined_data.append([match_result] + match_features)

# Convertimos la lista combinada en un DataFrame para su manipulación y visualización
column_names = ['Result'] + [f'Champ{i}_{feat}' for i in range(1, 11)
                             for feat in ['Difficulty', 'HP', 'Armor', 'HPRegen', 'AttackDamage', 'AttackSpeed']]
df_combined = pd.DataFrame(combined_data, columns=column_names)

# Mostramos las primeras filas del DataFrame para verificar
df_combined.head()

,Result,Champ1_Difficulty,Champ1_HP,Champ1_Armor,Champ1_HPRegen,Champ1_AttackDamage,Champ1_AttackSpeed,Champ2_Difficulty,Champ2_HP,Champ2_Armor,...,Champ9_Armor,Champ9_HPRegen,Champ9_AttackDamage,Champ9_AttackSpeed,Champ10_Difficulty,Champ10_HP,Champ10_Armor,Champ10_HPRegen,Champ10_AttackDamage,Champ10_AttackSpeed
0,1,8.0,630.0,33.0,8.5,64.0,0.625,7.0,655.0,27.0,...,29.835366,6.41311,58.920732,0.649543,3.0,610.0,47.0,8.5,55.0,0.644
1,0,8.0,596.0,22.0,5.5,55.0,0.625,7.0,600.0,20.0,...,28.000000,3.50000,50.000000,0.625000,5.0,575.0,27.0,9.0,61.0,0.645
2,1,4.0,650.0,40.0,6.0,64.0,0.658,5.0,670.0,38.0,...,29.000000,3.75000,62.000000,0.679000,5.0,610.0,30.0,5.0,62.0,0.635
3,0,3.0,660.0,35.0,8.0,69.0,0.665,7.0,658.0,30.0,...,29.835366,6.41311,58.920732,0.649543,5.0,610.0,30.0,5.0,62.0,0.635
4,0,3.0,660.0,35.0,8.0,69.0,0.665,8.0,558.0,19.0,...,24.000000,4.00000,62.000000,0.625000,4.0,646.0,35.0,8.5,68.0,0.644


In [3]:
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Separar los datos en variables independientes (X) y la variable dependiente (y)
X = df_combined.iloc[:, 1:].values  # Todas las filas, todas las columnas excepto la primera
y = df_combined.iloc[:, 0].values  # Todas las filas, solo la primera columna (resultado del partido)

# Normalizar las características
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
# Guardar el scaler para su uso posterior
joblib.dump(scaler, 'scaler.gz')
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Mostramos las dimensiones de los conjuntos de entrenamiento y prueba para confirmar
(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)


(((11026, 60), (11026,)), ((2757, 60), (2757,)))

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
import matplotlib.pyplot as plt



# Crear el modelo de árbol de decisión
clf = DecisionTreeClassifier(random_state=42)

# Entrenar el modelo
clf.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de entrenamiento
y_train_pred = clf.predict(X_train)

# Calcular el accuracy del conjunto de entrenamiento
accuracy_entrenamiento = accuracy_score(y_train, y_train_pred)

print(f"Accuracy del entrenamiento: {accuracy_entrenamiento:.2f}")

# Realizar predicciones sobre el conjunto de prueba
y_pred = clf.predict(X_test)

# Evaluar el modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy del conjunto de prueba: {accuracy:.2f}")


Accuracy del entrenamiento: 1.00
Accuracy del conjunto de prueba: 0.79


In [5]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, KFold

# Suponiendo que X y y ya están definidos
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # Define el método de validación cruzada

depths = range(1, 21)  # Rango de profundidades para explorar
mean_scores = []

for depth in depths:
    tree_model = DecisionTreeClassifier(max_depth=depth, random_state=42)
    scores = cross_val_score(tree_model, X, y, cv=kf, scoring='accuracy')  # Calcula la precisión para cada fold
    mean_scores.append(np.mean(scores))
    print(f"Profundidad {depth}: Precisión media = {np.mean(scores):.4f}, Desviación estándar = {np.std(scores):.4f}")

# Encontrar la profundidad con la mejor precisión media
optimal_depth = depths[np.argmax(mean_scores)]
print(f"La profundidad óptima es {optimal_depth} con una precisión media de {max(mean_scores):.4f}")


Profundidad 1: Precisión media = 0.5447, Desviación estándar = 0.0082
Profundidad 2: Precisión media = 0.5481, Desviación estándar = 0.0118
Profundidad 3: Precisión media = 0.5529, Desviación estándar = 0.0119
Profundidad 4: Precisión media = 0.5555, Desviación estándar = 0.0111
Profundidad 5: Precisión media = 0.5650, Desviación estándar = 0.0077
Profundidad 6: Precisión media = 0.5735, Desviación estándar = 0.0136
Profundidad 7: Precisión media = 0.5841, Desviación estándar = 0.0128
Profundidad 8: Precisión media = 0.5994, Desviación estándar = 0.0113
Profundidad 9: Precisión media = 0.6166, Desviación estándar = 0.0104
Profundidad 10: Precisión media = 0.6407, Desviación estándar = 0.0159
Profundidad 11: Precisión media = 0.6609, Desviación estándar = 0.0206
Profundidad 12: Precisión media = 0.6842, Desviación estándar = 0.0137
Profundidad 13: Precisión media = 0.7030, Desviación estándar = 0.0101
Profundidad 14: Precisión media = 0.7152, Desviación estándar = 0.0152
Profundidad 15: